In [ ]:
#If you are using google colab

# Do this
!pip install selenium
!apt-get update
!apt-get install chromium-chromedriver

In [3]:
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
import re

def club_img(soup):
  emblem = soup.find('img')['src']
  emblem_link = 'https://www.usashooting.org{}'.format(emblem)
  return emblem_link

def find_name(soup):
  name = soup.find('a')
  return name.text

def acronym_generator(soup):
  acronym = soup[0]
  for i in range(1, len(soup)):
    if soup[i-1] == ' ':
      acronym += soup[i]
  acronym = acronym.upper()
  return acronym

def find_website(soup):
  website = soup.find('a')
  return website['href']

def find_modalities(soup):
  modalities = soup.find('td', attrs = {'colspan': '2'})
  return modalities.text

def find_director(soup):
  director = soup.find_all('td', attrs = {'colspan': '2'})
  return director[1].text

def find_email(soup):
  try:
    email = soup.find_all('td', attrs = {'colspan': '2'})[2].find('a')
    return email.text.replace(' ', '')
  except:
    email = soup.find_all('td', attrs = {'colspan': '2'})[2].find('span')
    return email.text.replace(' ', '')

def find_phone(soup):
  phone = soup.find_all('td', attrs = {'colspan': '2'})[3].find('span')
  return phone.text

def find_address(soup):
  address = soup.find_all('td', attrs = {'colspan': '2'})[4].find('span')
  return address.text

def main():
  
  url = 'https://www.usashooting.org/about/certified-training-centers-20200707144939'
  
  agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('user-agent= {}'.format(agent))
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options = options)

  driver.get(url)
  selenium_get = driver.page_source
  soup = bs(selenium_get, 'html.parser')
  
  my_list = []

  info = soup.find_all('table', attrs = {'border': '5'})
  for i in info:
    my_dict = {}
    my_dict['Logo'] = club_img(i)
    my_dict['Name'] = find_name(i)
    my_dict['Acronym'] = acronym_generator(find_name(i))
    my_dict['Website'] = find_website(i)
    my_dict['Modalities'] = find_modalities(i)
    my_dict['Director'] = find_director(i)
    my_dict['Email'] = find_email(i)
    my_dict['Contact'] = find_phone(i)
    my_dict['Address'] = find_address(i)

    my_dict['Sport'] = 'Shooting'
    my_dict['Reach'] = 'City'
    my_dict['Category'] = 'Olympic, Paralympic'
    my_dict['Affilated'] = 'USA Shooting Certified Training Centers'
    my_dict['Recorded By'] = 'Eduardo Savio'
    my_dict['Recorded Date'] = '14/03/2022'

    my_list.append(my_dict)
  return my_list

res = main()

#save as xlsx
import pandas as pd
df = pd.DataFrame(data=res)
df.to_excel('USA_Shooting.xlsx')